## Adaptive Filter Attention

### Setup

In [ ]:
import numpy as np
import math
import scipy

from matplotlib import pyplot as plt
import matplotlib.cm as cm
plt.rcParams['figure.figsize'] = [10, 10]
plt.rc('font', size=20)

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import transformers
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
# from collections import Counter

import os
import argparse
import datetime
import time
from tqdm import tqdm # Loading bar
print('Done.')

In [ ]:
from utils import complex_exp, complex_hadamard, complex_matmul
from utils import batched_complex_exp, batched_complex_hadamard, batched_complex_matmul
from utils import batched_complex_hadamard_full, batched_complex_matmul_full
from utils import count_parameters, get_layers
print('Done.')

In [ ]:
from dynamics import stochastic_LTI, DynamicSim
from dynamics import construct_mapping
from dynamics import get_nth_measurement, get_random_measurements
from dynamics import linear_spiral, linear_spiral_3D, Lorenz, rand_coupling_matrix, Van_der_Pol_osc
print('Done.')

In [ ]:
from isotropic_afa import get_safe_exp_tot, compute_covariance_matrix, compute_covariance_matrix_safe
from isotropic_afa import compute_exp_kernel_isotropic, compute_exp_kernel_anisotropic
from isotropic_afa import build_factorized_kernels, compute_residual_norm_isotropic, compute_residual_norm_anisotropic

print('Done.')

In [ ]:
from model import compute_lambda, resolve_multihead_dims
from model import init_complexlinear, init_complex_matrix, set_complex_weight, initialize_linear_layers
from model import initialize_to_correct_model
from model import apply_weight_masks
from model import ComplexLinearLayer, ComplexLinearHermitianLayer, ComplextoRealLinearLayer
from model import MultiHeadAttentionLayer
from model import ComplexRMSNorm
from model import Attention_1layer, AFA_1layer
from model import SimpleAttention_Net
from model import TransformerBlock, TransformerNetwork
from model import MultiheadIsotropicAFA_1layer, AFATransformerBlock, AFATransformerNetwork
print('Done.')

In [ ]:
from data_utils import construct_random_mapping, construct_data_dynamics, TrainDataset_dynamics, create_train_loader_dynamics
print('Done.')

In [ ]:
from visualization import plot_trajectory, compute_state_matrix, plot_state_matrix, plot_eigenvals, visualize_results
from visualization import visualize_results_attn, _get_visual_modules
print('Done.')

In [ ]:
from training import single_iter, single_epoch, hook_fn
from training import single_iter_attn, single_epoch_attn
print('Done.')

In [ ]:
parser = argparse.ArgumentParser('DA')
parser.add_argument('--gpu', type=int, default=0) # (Default: 0)
args = parser.parse_args(args=[])
args.device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')
print(args.device)

torch.manual_seed(2025)
np.random.seed(2025)

### Training

In [ ]:
# Set dynamical system

# # Normal system
# D1 = torch.zeros(2,2,2).to(args.device) # Diagonal matrix
# D1[0] = torch.tensor([[-0.1, 0.0], [0.0, -0.1]]).to(args.device)
# D1[1] = torch.tensor([[-1.0, 0.0], [0.0, 1.0]]).to(args.device)
# # S1 = torch.zeros(2,2,2).to(args.device)
# # S1[0] = torch.tensor(([1.0,1.0],[0.0,0.0]))
# # S1[1] = torch.tensor(([0.0,0.0],[1.0,-1.0]))
# # S1 = U/np.sqrt(2)
# alpha = np.random.uniform(low=0.0, high=1.0)*2*np.pi
# beta = np.random.uniform(low=0.0, high=1.0)*2*np.pi
# S1 = construct_special_2D_unitary(alpha=alpha, beta=beta)
# Si1 = complex_conj_transpose(S1)

# Stable 2D linear system (in diagonalized form):
D1 = torch.zeros(2,2,2).to(args.device) # Diagonal matrix
S1 = torch.zeros(2,2,2).to(args.device) # RHS matrix
Si1 = torch.zeros(2,2,2).to(args.device) # Inverse of RHS matrix
D1[0] = torch.tensor([[-0.1, 0.0], [0.0, -0.1]]).to(args.device) # Negative real part eigenvals
# D1[0] = torch.tensor([[0.0, 0.0], [0.0, 0.0]]).to(args.device) # Zero real part eigenvals
# D1[0] = torch.tensor([[0.1, 0.0], [0.0, 0.1]]).to(args.device) # Positive real part eigenvals
D1[1] = torch.tensor([[-1.0, 0.0], [0.0, 1.0]]).to(args.device)
S1[0] = torch.tensor([[1.0, 1.0], [1.0, 1.0]]).to(args.device)
S1[1] = torch.tensor([[-1.0, 1.0], [0.0, 0.0]]).to(args.device)
Si1[0] = 0.5*torch.tensor([[0.0, 1.0], [0.0, 1.0]]).to(args.device)
Si1[1] = 0.5*torch.tensor([[1.0, -1.0], [-1.0, 1.0]]).to(args.device)
A = complex_matmul(S1,complex_matmul(D1,Si1))[0].unsqueeze(0)
params = [D1, S1, Si1]

In [ ]:
# DEFINE MODEL

args.cr_max = 2
args.cr_min = 0

args.max_exponent = 10
args.min_exponent = -10

args.t_equal = 1 # Equal time intervals? (0 or 1)
args.sep_params = 0 # Use separate params for keys and values? (0 or 1)

##################################################
## Testing options
args.rand_embed = 0 # Use random orthogonal matrix when projecting data to higher dimension?
args.weight_mask = 0 # Mask the weights? (0 or 1) (must be set to 1 when visualizing in 2D)
args.lambda_real_zero = 0 # Force real part of lambda to be zero? (0=No, 1=Yes)
##################################################

##################################################
## Ablation options ##
######################
## (Setting everything to 0 (except lambda_real_zero & outer residual norm) makes it an ordinary Transformer)
args.lambda_real_zero = 0 # Zero out real part of eigenvalues? (1 = yes, 0 = no)
args.use_full_residual_norm = 1 # Use the full |R|^2 metric and rational robust weight (1) or dot product / exponential weight (0)?
args.use_robust_weight = 1 # Use rational weight rather than softmax
args.additive_bias_type = 1 # (Additive bias: 0 for zero; 1 for DLE; 2 for linear)
args.multiplicative_bias_type = 1 # (Multiplicative bias: 0 for constant; 1 for DLE; 2 for linear)
args.compute_next_step_pred = 1 # Compute next-step prediction? (1 = yes, 0 = No)
args.learn_rotations = 1 # Learned rotations (1), or fixed as in RoPE (0)?
args.rotate_values = 1 # Rotate/unrotate values? (1 = yes, 0 = No)
args.zero_process_noise = 0 # Zero process noise (sigma^2)? (1 = yes, 0 = No)
args.zero_key_measurement_noise = 0 # Zero key measurement noise (eta^2)? (1 = yes, 0 = No)
args.use_total_precision_gate = 0 # Use total-precision gating? (0 = No gate, 2 = precision gate, 2 = learned gate)
args.use_complex_input_norm = 0 # Use complex-valued RMS Norm AFTER input projection (1) or real-valued RMS Norm BEFORE input projection (0)?
args.use_inner_residual_and_norm = 0
# args.use_complex_output_norm = 1
# args.use_inner_residual = 1
# args.use_outer_residual = 0
args.add_gaussian_noise = 0 # Add Gaussian noise (for test-time sampling)

##################################################

args.compute_next_step_pred = 1

args.m = 2 # Dimension of simulated system
# args.embed_dim = 2 # Embedding dimension
args.d_e = 128 # Embedding dimension

# args.num_heads = 1
# args.head_dim = int(args.embed_dim/args.num_heads)

##################################################

# Key, query, value embedding dimensions are same as input embedding dimension
args.d_k = args.d_e
args.d_v = args.d_e
# args.d_k = args.head_dim
# args.d_v = args.head_dim

# # Key, query, value embedding dimensions are half of input embedding dimension
# args.d_k = int(args.head_dim/2) # Key and query embedding dimension
# args.d_v = int(args.head_dim/2) # Value embedding dimension

# # Key, query, value embedding dimensions are 2
# args.d_k = 2 # Key and query embedding dimension
# args.d_v = 2 # Value embedding dimension

##################################################

args.nu = 1.0 # Measurement weighting
args.tf = 10.0 # Final time
args.dt = 0.01 # Time step size
args.n = 10 # nth measurement (ie use every nth point as a measurement)
args.delta_t = args.dt * args.n

args.N_t = int(args.tf/args.dt) # Number of time steps
args.seq_len = int(args.N_t/args.n)+1 # Number of measurements
# args.seq_len = args.N_t / args.n = (args.tf/args.dt) / (args.delta_t/args.dt) = args.tf/args.delta_t
t_v = (torch.arange(args.N_t + args.n + 1)*args.dt).to(args.device) # Array of time steps

# Some scalar weights in model
args.alpha = 1.0
args.beta = 0.0
args.delta = 1.0
args.eta = 0.0

In [ ]:
# DEFINE MODEL:

###########################################################

# # Standard Attention/Transformer Models

# args.model_type = 'RealInputs'
# args.num_heads = 4
# args.num_blocks = 3
# # model = Attention_1layer(args.d_e, args.d_v, args.num_heads, args).to(args.device)
# # model = SimpleAttention_Net(args.d_e, args.d_v*2, args.num_heads, args).to(args.device)
# model = TransformerNetwork(args.d_e, args.d_v*2, args.d_v*4, args.num_heads, args, num_blocks=args.num_blocks).to(args.device)
# loss = nn.MSELoss()
# loss_p = nn.MSELoss()

###########################################################

# # Complex Attention with Real Input/Output Models

# args.model_type = 'RealInputs'
# args.metric = 'RealDotProduct'
# # args.metric_type = 'InverseMahalanobis'
# # model = SimpleComplexAttention_Net(args.d_e, args.d_v, args).to(args.device)
# model = ComplexTransformerNetwork(args.d_e, args.d_v*2, args.d_v*4, args, num_blocks=2).to(args.device)
# loss = nn.MSELoss()

###########################################################

# # Multihead Isotropic AFA

# args.num_heads = 2 # Number of heads
# # args.model_type = 'RealInputs' # Real-valued inputs
# # args.num_inner_layers = 1 # Number of layers inside an Nlayer block
# # self.args.compute_next_step_pred = 0
# # args.d_k = args.d_e
# # args.d_v = args.d_e
# # model = MultiheadSimplifiedAFA_1layer(args, args.seq_len, args.num_heads, input_dim=args.d_e, query_key_dim=args.d_k, value_dim=args.d_v).to(args.device)
# args.d_k_total = args.d_e # Total query-key dim across all heads
# args.d_v_total = args.d_e # Total value dim across all heads
# model = MultiheadIsotropicAFA_1layer(args, args.num_heads, input_dim=args.d_e, query_key_dim_total=args.d_k_total, value_dim_total=args.d_v_total).to(args.device)

# # args.num_blocks = 1 # Number of transformer blocks
# # model = MultiheadSimplifiedAFATransformerNetwork(args, args.seq_len, num_blocks=args.num_blocks, n_heads=args.num_heads, input_dim=args.d_e, query_key_dim_total=args.d_k_total, value_dim_total=args.d_v_total, Norm=Norm).to(args.device)

# loss = nn.MSELoss() # Loss
# loss_p = nn.MSELoss() # Penalty

###########################################################

# AFA Transformer

args.compute_next_step_pred = 0
args.num_heads = 2 # Number of heads
args.num_blocks = 2 # Number of transformer blocks
args.d_k_total = args.d_e # Total query-key dim across all heads
args.d_v_total = args.d_e # Total value dim across all heads
model = AFATransformerNetwork(args=args, num_blocks=args.num_blocks, n_heads=args.num_heads, input_dim=args.d_e, query_key_dim_total=args.d_k, value_dim_total=args.d_v, hidden_dim = 4*args.d_v, Norm=nn.LayerNorm).to(args.device)

loss = nn.MSELoss()
loss_p = nn.MSELoss()

###########################################################

In [ ]:
params_list = list(model.parameters()) # Parameters list

# loss_p = Complex_Trace_Loss() # Trace Penalty
lr = 1E-2 # Learning rate
optimizer = torch.optim.Adam(params_list, lr=lr, betas=(0.9, 0.999)) # Optimizer

if args.d_k != args.d_v and args.sep_params == 0:
    print('ERROR: Key and value embedding dimensions must be the same if using shared parameters.')

print('Total parameter count:', count_parameters(model))

#####################

# Create folders for model weights, and loss history

saved_models_path = 'C:/Users/Pracioppo/Desktop/AFA, Dec 11/saved_models/'
model_name = str(model.__class__.__name__)
date = str(datetime.datetime.today()).split()[0]
model_path = saved_models_path + model_name + '__' + date + '/'

model_weight_path = model_path + 'model_weights/'
model_tensor_path = model_path + 'info_tensors/'

try:
    os.mkdir(model_path)
    os.mkdir(model_weight_path)
    os.mkdir(model_tensor_path)
except:
    pass

print('Model loaded.')

In [ ]:
## CREATE TRAINING DATA

args.concat_mag = 0

# Training params
args.num_epochs = 1000 # Number of epochs
args.num_samp = 32 # Number of samples in train loader
args.batch_size = 32 # Batch size
args.num_its = int(args.num_samp/args.batch_size) # Number of iterations in an epoch
args.save_epochs = 100 # Intervals of epochs to save model
args.show_example_epochs = 5 # Number of epochs between displaying results so far
args.n_example = 5 # Plot state estimates at n_example data points
args.epsilon = 1E-5

# Process and measurement noise:
sigma_process = 0.0 # Process noise
sigma_measure = 2.0 # Measurement noise

#####################################################################

# Initialize arrays to record losses
mean_epoch_losses = np.zeros(args.num_epochs)
log_mean_epoch_losses = np.zeros(args.num_epochs)
all_losses = np.zeros(args.num_epochs * args.num_samp)
all_lambdas = np.zeros((args.num_epochs, 2, args.d_v))

Pu, Pd, R1, R1i = construct_random_mapping(S1, Si1, args) # Get random matrices

# Build training dataset
train_loader, train_dataset, X_true_all, X_measure_all, t_measure_all = create_train_loader_dynamics(A, S1, Si1, Pu, Pd, R1, R1i, t_v, sigma_process, sigma_measure, args)

In [ ]:
# PLOT ALL TRAJECTORIES

from matplotlib import patches
fig, ax = plt.subplots()

for it, (train_data, X_true, X_measure, t_measure_full) in enumerate(train_loader):
# for it, train_data in enumerate(train_loader):
#     for i in range(args.batch_size):

#     if X_true.size()[2] == 1:
#         X_true = torch.stack((X_true, torch.zeros_like(X_true)),dim=2)
#         X_measure = torch.stack((X_measure, torch.zeros_like(X_measure)),dim=2)

    # Actual trajectory
    X_true_plt = X_true.squeeze().detach().cpu().numpy()
    plt.plot(X_true_plt.T[0],X_true_plt.T[1],'black')

    # Noisy trajectory
    traj = X_measure.detach().cpu().squeeze().numpy()
    plt.plot(traj.T[0], traj.T[1], 'b--')
    
    x0 = X_true_plt[:,0].T
    plt.scatter(x0[0],x0[1], color='red')

#     circle = patches.Circle((0,0), 17.0, edgecolor='red', facecolor='none', linewidth = 2)
#     ax.add_patch(circle)

    plt.axis('equal')
    plt.grid()
    plt.show()

In [ ]:
# ###### Initialize model to correct values (for testing) ######
# ##############################################################

# if isinstance(model, MultiheadIsotropicAFA_1layer):
#     module = model.layers[0]
#     initialize_to_correct_model(module, D1, S1, Si1, sigma_process, sigma_measure, args)

In [ ]:
######  Load in model weights ######
####################################

# epoch = 410
# model_weight_path_full = model_weight_path + 'weights_epoch_' + str(epoch)

# try:
#     model.load_state_dict(torch.load(model_weight_path_full, weights_only=True))
#     print('Model loaded.')
# except:
#     print('Model loading failed.')
#     pass

In [ ]:
########################### TRAINING LOOP ###########################
#####################################################################

###### LOOP FOR PRECISION ATTENTION LAYERS #######

# Train for num_epochs:
for epoch in tqdm(np.arange(args.num_epochs), desc="Training progress..."):

    # Train for single epoch
    epoch_losses, A_hat_complex, epoch_lambdas, unnormalized_attention = single_epoch(model, train_loader, optimizer, loss, loss_p, params_list, args)

    # Collect losses
    all_losses[epoch*args.num_its:(epoch+1)*args.num_its] = epoch_losses
    mean_epoch_losses[epoch] = np.mean(epoch_losses)
    log_mean_epoch_losses[epoch] = np.log(np.mean(epoch_losses))
    all_lambdas[epoch,:,:] = np.mean(epoch_lambdas,axis=0)
    
    # Visualize results so far:
    if np.mod(epoch+1,args.show_example_epochs) == 0:
        visualize_results(model, train_dataset, all_losses, mean_epoch_losses, log_mean_epoch_losses, all_lambdas, R1, R1i, Pu, Pd, A, epoch, args)

#     if np.mod(epoch+1,args.save_epochs) == 0:
#         model_weight_path_full = model_weight_path + 'weights_epoch_' + str(epoch)
#         model_tensor_path_full = model_tensor_path + 'tensors_epoch_' + str(epoch)
#         torch.save(model.state_dict(), model_weight_path_full)

#         tensors_to_save = {
#         'epoch_losses': epoch_losses,
#         'A_hat_complex': A_hat_complex,
#         'epoch_lambdas': epoch_lambdas
#         }
#         torch.save(tensors_to_save, model_tensor_path_full)

In [ ]:
# # Save Model
# model_weight_path_full = model_weight_path + 'weights_epoch_' + str(epoch)
# model_tensor_path_full = model_tensor_path + 'tensors_epoch_' + str(epoch)
# torch.save(model.state_dict(), model_weight_path_full)

# tensors_to_save = {
# 'epoch_losses': epoch_losses,
# 'A_hat_complex': A_hat_complex,
# 'epoch_lambdas': epoch_lambdas
# }
# torch.save(tensors_to_save, model_tensor_path_full)

In [ ]:
# ##### LOOP FOR REAL/COMPLEX ATTENTION NETS #######

# for epoch in tqdm(np.arange(args.num_epochs), desc="Training progress..."):

#     epoch_losses = single_epoch_attn(model, train_loader, optimizer, loss, params_list, args)

#     # Collect losses
#     all_losses[epoch*args.num_its:(epoch+1)*args.num_its] = epoch_losses
#     log_mean_epoch_losses[epoch] = np.log(np.mean(epoch_losses))
    
#     # Visualize results so far:
#     if np.mod(epoch+1,args.show_example_epochs) == 0:
#         visualize_results_attn(model, train_dataset, all_losses, mean_epoch_losses, log_mean_epoch_losses, all_lambdas, R1, R1i, Pu, Pd, A, epoch, args)
# ##################################################

In [ ]:
# Sample the model autoregressively
def autoregressive_sample(model, start_seq, max_gen_len):

    with torch.no_grad():
        model.eval()
        current_seq = start_seq
        total_seq = start_seq
        
        window_size = start_seq.size(2)

        for i in range(max_gen_len):
            # Forward pass through the model
            out, output_dict = model(current_seq)

            # Extract the last generated token (the prediction for the next step)
            next_token = out[:, :, -1].unsqueeze(2)

            # Append new token
            total_seq = torch.cat([total_seq, next_token], dim=2)

            # Remove first token to keep context window fixed
            current_seq = total_seq[:,:,-window_size:]

    new_seq = total_seq[:,:,-args.max_gen_len:]

    return total_seq, new_seq
    
args.add_gaussian_noise = 1
args.max_gen_len = 100

for it, (train_data, _, _, _) in enumerate(train_loader):
    start_seq = train_data[:, :-1].unsqueeze(1)
    break
    
total_seq, new_seq = autoregressive_sample(model, start_seq, args.max_gen_len)
    
traj_tot = torch.matmul(R1i,total_seq.unsqueeze(-1)) # Reverse random mapping
X_tot = torch.matmul(Pd,traj_tot) # Map back to lower dim

traj_new = torch.matmul(R1i,new_seq.unsqueeze(-1)) # Reverse random mapping
X_new = torch.matmul(Pd,traj_new) # Map back to lower dim

# Predicted trajectory
X_tot_plt = X_tot.squeeze(0)[0].detach().cpu().squeeze().numpy()
plt.plot(X_tot_plt.T[0], X_tot_plt.T[1], 'r--', label='Predicted') # Added label

X_new_plt = X_new.squeeze(0)[0].detach().cpu().squeeze().numpy()
plt.plot(X_new_plt.T[0], X_new_plt.T[1], 'b', label='Predicted') # Added label

plt.grid()